## Deep Learning Model: Chemprop 

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import chemprop
from sklearn.utils import shuffle
from chemprop.train.evaluate import evaluate_predictions


## Random Sampling 

In [12]:
def run_chemprop(batchsize):
    train_arguments = [
        '--data_path', '/Users/shwetajones/Desktop/1. Automation/02-750-Final-Project/rand_train_dataset.csv',
        '--dataset_type', 'regression',
        '--save_dir', '/Users/shwetajones/Desktop/1. Automation/02-750-Final-Project/',
        '--batch_size', str(batchsize), 
        '--epochs', str(5),
        
    ]

    train_args = chemprop.args.TrainArgs().parse_args(train_arguments)
    train_args.quiet = True
    mean_score, std_score = chemprop.train.cross_validate(args=train_args, train_func=chemprop.train.run_training)

    test_arguments = [
    '--test_path', '/Users/shwetajones/Desktop/1. Automation/02-750-Final-Project/rand_test_dataset.csv',
    '--preds_path', 'rand_preds.csv',
    '--checkpoint_dir', '/Users/shwetajones/Desktop/1. Automation/02-750-Final-Project/', 
    
    ]   

    test_args = chemprop.args.PredictArgs().parse_args(test_arguments)
    test_args.quiet = True
    preds = chemprop.train.make_predictions(args=test_args)

    preds = pd.read_csv("rand_preds.csv")
    y_preds = [[target] for target in preds["target"].values]
    test = pd.read_csv("rand_test_dataset.csv")
    y_test = [[target] for target in test["target"].values]
    dataset_type = 'test'
    num_tasks = len(y_preds[0])
    results = evaluate_predictions(preds=y_preds, targets=y_test, num_tasks=num_tasks, metrics=['rmse', 'r2'], dataset_type=dataset_type)
  
    return results['rmse'], results['r2']

In [2]:
def random_sampling_chemprop(batchsize):
    seeds = [1, 2, 3]
    
    rmse_vals = []
    r2_vals = []

    data = pd.read_csv("TYK2_final.csv", index_col=False)
    data = data.drop(['target', 'top_2p', 'top_5p'], axis=1)
    column_names = ['smiles', 'target']
    data.columns = column_names

    for seed in seeds:
        rmse_instance = []
        r2_instance = []
        # Get the data 
        data = np.array(data)
        shuffled_data = shuffle(data, random_state=seed)
        size = int(len(shuffled_data)*0.1)
        train, test = shuffled_data[:size], shuffled_data[size:]
        train_df = pd.DataFrame(train, columns=column_names)
        train_df.to_csv("rand_train_dataset.csv", index=False)
        test_df = pd.DataFrame(test, columns=column_names)
        test_df.to_csv("rand_test_dataset.csv", index=False)
        
        counter = 0
        while len(train_df) <= round(((len(shuffled_data) * 0.2) / 500 )) * 500:
            if counter % 100 == 0:
                print("running chemprop")
                rmse, r2 = run_chemprop(batchsize) 
                rmse_instance.append(rmse)
                r2_instance.append(r2)
            train_df =  pd.concat([train_df, test_df.iloc[:batchsize]])  
            test_df = test_df.drop(test_df.index[:batchsize]) 
            
            # Update the csv
            train_df.to_csv("rand_train_dataset.csv", index=False)
            test_df.to_csv("rand_test_dataset.csv", index=False)
            
            counter += batchsize
        
        rmse_vals.append(rmse_instance)
        r2_vals.append(r2_instance)
        print("finished seed")
    return rmse_vals, r2_vals


In [ ]:
batch_sizes = [1, 25, 50]
rand_rmse_dict = {}
rand_r2_dict = {}

final_rand_data_chemprop = pd.DataFrame()

for batch in batch_sizes:
    print(f"{batch} number")
    rand_rmse, rand_r2 = random_sampling_chemprop(batch)

    rand_rmse_mean = np.mean(rand_rmse, axis=0)
    rand_rmse_stdev = np.std(rand_rmse, axis=0)
    print("mean:", rand_rmse_mean)
    print("standard deviation:", rand_rmse_stdev)
    rand_rmse_dict[batch] = (rand_rmse, rand_rmse_mean, rand_rmse_stdev)
    final_rand_data_chemprop[f"{batch}_rmse_mean"] = rand_rmse_mean
    final_rand_data_chemprop[f"{batch}_rmse_stdev"] = rand_rmse_stdev


    rand_r2_mean = np.mean(rand_r2, axis=0)
    rand_r2_stdev = np.std(rand_r2, axis=0)
    print("mean:", rand_r2_mean)
    print("standard deviation:", rand_r2_stdev)
    rand_r2_dict[batch] = (rand_r2, rand_r2_mean, rand_r2_stdev)
    final_rand_data_chemprop[f"{batch}_r2_mean"] = rand_r2_mean
    final_rand_data_chemprop[f"{batch}_r2_stdev"] = rand_r2_stdev
    

final_rand_data_chemprop.to_csv("final_data_random_chemprop.csv", index=False)

## Monte Carlo Dropout

In [2]:
def chemprop_monte_carlo(batchsize, eval):
    train_arguments = [
        '--data_path', '/Users/shwetajones/Desktop/1-Automation/02-750-Final-Project/mc_train_dataset.csv',
        '--dataset_type', 'regression',
        '--save_dir', '/Users/shwetajones/Desktop/1-Automation/02-750-Final-Project/',
        '--batch_size', str(batchsize), 
        '--loss_function', 'mve', 
        '--epochs', str(5),
        
    ]

    train_args = chemprop.args.TrainArgs().parse_args(train_arguments)
    train_args.quiet = True
    mean_score, std_score = chemprop.train.cross_validate(args=train_args, train_func=chemprop.train.run_training)

    test_arguments = [
    '--test_path', '/Users/shwetajones/Desktop/1-Automation/02-750-Final-Project/mc_test_dataset.csv',
    '--preds_path', 'mc_preds.csv',
    '--checkpoint_dir', '/Users/shwetajones/Desktop/1-Automation/02-750-Final-Project/', 
    '--uncertainty_dropout_p', str(0.1), 
    '--uncertainty_method', 'mve',
    ]   

    test_args = chemprop.args.PredictArgs().parse_args(test_arguments)
    test_args.quiet = True
    preds = chemprop.train.make_predictions(args=test_args, return_uncertainty=True)

    preds = pd.read_csv("mc_preds.csv")
    uncertainty = preds['target_mve_uncal_var']
    instance_to_add = uncertainty.nlargest(batchsize).index
    
    if eval == True:
        y_preds = [[target] for target in preds["target"].values]
        test = pd.read_csv("mc_test_dataset.csv")
        y_test = [[target] for target in test["target"].values]
        dataset_type = 'test'
        num_tasks = len(y_preds[0])
        print("INFORMATIONS:", len(y_preds), len(y_test), num_tasks)
        results = evaluate_predictions(preds=y_preds, targets=y_test, num_tasks=num_tasks, metrics=['rmse', 'r2'], dataset_type=dataset_type)
        return instance_to_add, results['rmse'], results['r2']
    
    return instance_to_add
  
    

In [3]:
def monte_carlo_dropout(batchsize):
    seeds = [1, 2, 3]
    
    rmse_vals = []
    r2_vals = []

    data = pd.read_csv("TYK2_final.csv", index_col=False)
    data = data.drop(['target', 'top_2p', 'top_5p'], axis=1)
    column_names = ['smiles', 'target']
    data.columns = column_names

    for seed in seeds:
        rmse_instance = []
        r2_instance = []
        # Get the data 
        data = np.array(data)
        shuffled_data = shuffle(data, random_state=seed)
        # size = int(len(shuffled_data)*0.1)
        size = 1000
        train, test = shuffled_data[:size], shuffled_data[size:]
        train_df = pd.DataFrame(train, columns=column_names)
        train_df.to_csv("mc_train_dataset.csv", index=False)
        test_df = pd.DataFrame(test, columns=column_names)
        test_df.to_csv("mc_test_dataset.csv", index=False)
        print("TEST/TRAIN:", len(test_df), len(train_df))
        

        counter = 0
        eval = False
        while len(train_df) <= 1500:#round(((len(shuffled_data) * 0.125) / 50 )) * 50:
            print("running chemprop")
            if counter % 50 == 0:
                print("in the counter")
                eval = True 
                instance, rmse, r2 = chemprop_monte_carlo(batchsize, eval) 
                rmse_instance.append(rmse)
                r2_instance.append(r2)
            
            else:
                instance = chemprop_monte_carlo(batchsize, eval) 

            eval = False

            train_df =  pd.concat([train_df, test_df.iloc[instance]])  
            test_df = test_df.drop(test_df.index[instance]) 
    
            # Update the csv
            train_df.to_csv("mc_train_dataset.csv", index=False)
            test_df.to_csv("mc_test_dataset.csv", index=False)

            # test = pd.read_csv("mc_test_dataset.csv", index_col=False)
            # train = pd.read_csv("mc_train_dataset.csv", index_col=False)
            # print("UPDATED TEST/TRAIN:", len(test), len(train))

            counter += batchsize
        
        rmse_vals.append(rmse_instance)
        r2_vals.append(r2_instance)
        print("finished seed")
    return rmse_vals, r2_vals

In [4]:
batch_sizes = [25]
mc_rmse_dict = {}
mc_r2_dict = {}

final_mc_data_chemprop = pd.DataFrame()

for batch in batch_sizes:
    print(f"{batch} number")
    mc_rmse, mc_r2 = monte_carlo_dropout(batch)

    mc_rmse_mean = np.mean(mc_rmse, axis=0)
    mc_rmse_stdev = np.std(mc_rmse, axis=0)
    print("mean:", mc_rmse_mean)
    print("standard deviation:", mc_rmse_stdev)
    mc_rmse_dict[batch] = (mc_rmse, mc_rmse_mean, mc_rmse_stdev)
    # final_mc_data_chemprop[f"{batch}_rmse_mean"] = mc_rmse_mean
    # final_mc_data_chemprop[f"{batch}_rmse_stdev"] = mc_rmse_stdev


    mc_r2_mean = np.mean(mc_r2, axis=0)
    mc_r2_stdev = np.std(mc_r2, axis=0)
    print("mean:", mc_r2_mean)
    print("standard deviation:", mc_r2_stdev)
    mc_r2_dict[batch] = (mc_r2, mc_r2_mean, mc_r2_stdev)
    # final_mc_data_chemprop[f"{batch}_r2_mean"] = mc_r2_mean
    # final_mc_data_chemprop[f"{batch}_r2_stdev"] = mc_r2_stdev
print(mc_rmse_dict)
print(mc_r2_dict)
    

# final_mc_data_chemprop.to_csv("final_data_mc_chemprop.csv", index=False)

25 number
TEST/TRAIN: 8997 1000
running chemprop
in the counter


1000it [00:00, 203123.83it/s]
100%|██████████| 1000/1000 [00:00<00:00, 4341.07it/s]
Fold 0
100%|██████████| 5/5 [00:16<00:00,  3.30s/it]
Model 0 best validation rmse = 1.222764 on epoch 4
Model 0 test rmse = 1.382127                 
Ensemble test rmse = 1.382127
1-fold cross validation
	Seed 0 ==> test rmse = 1.382127
Overall test rmse = 1.382127 +/- 0.000000
Elapsed time = 0:00:18


Loading training args
Setting molecule featurization parameters to default.
Loading data


8997it [00:00, 21857.16it/s]
100%|██████████| 8997/8997 [00:00<00:00, 174672.07it/s]


Validating SMILES
Test size = 8,997


...

Validating SMILES
Test size = 8,497


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".


100%|██████████| 1/1 [05:05<00:00, 305.24s/it]

Saving predictions to mc_preds.csv
Elapsed time = 0:05:06
INFORMATIONS: 8497 8497 1
finished seed
mean: [[1.23512409]
 [1.22528164]
 [1.21532052]
 [1.1819137 ]
 [1.20218246]
 [1.17442579]
 [1.19362385]
 [1.23972294]
 [1.16395753]
 [1.18629126]
 [1.17807874]]
standard deviation: [[0.01947302]
 [0.03437509]
 [0.01997922]
 [0.01139693]
 [0.02173293]
 [0.03159526]
 [0.00281188]
 [0.01993995]
 [0.02286761]
 [0.02055482]
 [0.03780976]]
mean: [[0.17683468]
 [0.17609466]
 [0.17821285]
 [0.21790788]
 [0.18563873]
 [0.21777841]
 [0.18945314]
 [0.1212657 ]
 [0.22211347]
 [0.189957  ]
 [0.19742237]]
standard deviation: [[0.0250063 ]
 [0.04491036]
 [0.02422036]
 [0.01145824]
 [0.02907253]
 [0.03940501]
 [0.00800625]
 [0.02200529]
 [0.02834789]
 [0.0298981 ]
 [0.051804  ]]
{25: ([[[1.2193025978244991], [1.192648582287506], [1.1910054741431748], [1.1724275290349646], [1.1798569772243448], [1.1354613708081427], [1.1932672690387867], [1.2326544550116447], [1.1342876890439153], [1.1701759541127914], [1.

In [5]:
print(mc_rmse_dict)
print(mc_r2_dict)

{25: ([[[1.2193025978244991], [1.192648582287506], [1.1910054741431748], [1.1724275290349646], [1.1798569772243448], [1.1354613708081427], [1.1932672690387867], [1.2326544550116447], [1.1342876890439153], [1.1701759541127914], [1.144791398916226]], [[1.2235141312762203], [1.2103926904766562], [1.2150145838635031], [1.1753720819022802], [1.19505148492098], [1.2128479438204796], [1.1903721712289461], [1.2668988818881264], [1.1676494295516053], [1.1733972033085938], [1.1584825090757997]], [[1.2625555450860093], [1.2728036617758607], [1.2399414964477562], [1.1979414829230768], [1.2316389237608072], [1.1749680435658185], [1.1972321033527278], [1.2196154820851468], [1.189935468803972], [1.2153006124679655], [1.230962324515671]]], array([[1.23512409],
       [1.22528164],
       [1.21532052],
       [1.1819137 ],
       [1.20218246],
       [1.17442579],
       [1.19362385],
       [1.23972294],
       [1.16395753],
       [1.18629126],
       [1.17807874]]), array([[0.01947302],
       [0.03